<a href="https://colab.research.google.com/github/TANZHIJIAN14/A-E-Chatbot/blob/main/A%26E_chatbot__fine_tuning_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Util function and variable

In [62]:
import pandas as pd
import numpy as np

uncleaned_dataset_path = '/content/sample_data/dataset/'
cleaned_dataset_path = '/content/sample_data/cleaned_dataset/'
tokenized_dataset_path = '/content/sample_data/tokenized_dataset/'

def replace_empty_value(file_path):
    df = pd.read_csv(file_path)
    # Replace empty strings and other missing value indicators with NaN
    df.replace("", np.nan, inplace=True)
    return df

def remove_columns(file_path, columns_to_remove):
    print('Called')
    df = pd.read_csv(file_path)

    # Remove the specified columns
    for column in columns_to_remove:
        if column in df.columns:
            df = df.drop(columns=[column])

    return df

def rename_columns(file_path, column_mapping):
    df = pd.read_csv(file_path)

    # Iterate over each key-value pair in the column mapping dictionary
    for old_name, new_name in column_mapping.items():
        # Check if the old column name exists before renaming
        if old_name in df.columns:
            # Rename the column
            df.rename(columns={old_name: new_name}, inplace=True)

    return df

def concat_files(checkFile):
    # List to hold DataFrames
    dfs = []

    # Loop through each CSV file in the directory
    for filename in all_processed_files_path:
        # Read the CSV file into a DataFrame and append it to the list
        to_be_combine_file = filename.split('/')[-1][0:12]
        if to_be_combine_file == checkFile:
          df = pd.read_csv(filename)
          dfs.append(df)

          #Remove the file
          if os.path.exists(filename):
            os.remove(filename)

    # Concatenate all DataFrames in the list along the rows axis (axis=0)
    if len(dfs) > 0:
      return pd.concat(dfs, axis=0, ignore_index=True)
    else:
      return None


# Load file paths

In [63]:
import os
import glob

all_files_path = glob.glob(os.path.join(uncleaned_dataset_path, "*.csv"))
all_processed_files_path = glob.glob(os.path.join(cleaned_dataset_path, "*.csv"))

#'/content/dataset/guideLine.csv'
#split -> 'guideLine.csv'
all_files_name = [file.split('/')[-1] for file in all_files_path]

# Replace the empty value with NaN

In [64]:
# List to store cleaned DataFrames
cleaned_dfs = []

for file_path in all_files_path:
    cleaned_df = replace_empty_value(file_path)
    cleaned_dfs.append(cleaned_df)

# Save each cleaned DataFrame individually and overwrite the original dataset
for i, df in enumerate(cleaned_dfs):
    df.to_csv(cleaned_dataset_path + all_files_name[i], index=False)

# Remove columns

In [ ]:
import pandas as pd

#Name of column to be removed
columns_to_remove = ['web-scraper-order', 'web-scraper-start-url']

# List to store cleaned DataFrames
cleaned_dfs = []

for file_path in all_processed_files_path:
    cleaned_df = remove_columns(file_path, columns_to_remove)
    cleaned_dfs.append(cleaned_df)


# Save each cleaned DataFrame individually and overwrite the original dataset
for i, df in enumerate(cleaned_dfs):
    df.to_csv(cleaned_dataset_path + all_files_name[i], index=False)

# Rename column

In [66]:
# Define a dictionary of old column names mapped to new column names
column_mapping = {
    'link': 'linkName',
    'link-href': 'link',
    'paymentLink': 'paymentName',
    'paymentLink-href': 'paymentLink',
    'eventLink': 'eventName',
    'eventLink-href': 'eventLink',}

# List to store cleaned DataFrames
cleaned_dfs = []
for file_path in all_processed_files_path:
    cleaned_df = rename_columns(file_path, column_mapping)
    cleaned_dfs.append(cleaned_df)

# Save each cleaned DataFrame individually and overwrite the original dataset
for i, df in enumerate(cleaned_dfs):
    df.to_csv(cleaned_dataset_path + all_files_name[i], index=False)

# Combine and remove csv files

In [67]:
combined_df = concat_files('visitingHour')

if combined_df is not None:
  # Save the combined DataFrame to a new CSV file
  combined_file_path = cleaned_dataset_path + 'allVisitingHour.csv'
  combined_df.to_csv(combined_file_path, index=False)

# Reset the all cleaned files path
all_processed_files_path = glob.glob(os.path.join(cleaned_dataset_path, "*.csv"))

# Install dependencies

In [8]:
# Need to run this once per machine
# !pip install -q -U bitsandbytes
# !pip install -q -U transformers
# !pip install -q -U accelerate
# !pip install -q -U git+https://github.com/huggingface/peft.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


# Load Tokenizer Model

In [2]:
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
)

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
#Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id =  tokenizer.unk_token_id
tokenizer.padding_side = 'left'

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

# Formatting prompt

In [69]:
def formatting_func(example):
    text = f"### The following is a doctor's opinion on a person's query: \n### Patient query: {example['input']} \n### Doctor opinion: {example['output']}"
    return text

# Tokenization dataset setup

In [3]:
import pandas as pd
from transformers import AutoTokenizer
from sklearn.model_selection import train_test_split

max_length = 512 # differs from datasets to datasets

def tokenize_dataset(file):
  # Load the CSV file into a pandas DataFrame
  df = pd.read_csv(file)

  # Function to tokenize a single text column
  def tokenize_column(series, tokenizer):
      return series.apply(lambda x: tokenizer(str(x), padding='max_length', truncation=True, return_tensors='pt'))

  # Apply the tokenizer to all columns
  tokenized_data = {col: tokenize_column(df[col], tokenizer) for col in df.columns}

  # Combine tokenized columns into the DataFrame
  for col in df.columns:
      df[col + '_input_ids'] = tokenized_data[col].apply(lambda x: x['input_ids'].squeeze().tolist())
      df[col + '_attention_mask'] = tokenized_data[col].apply(lambda x: x['attention_mask'].squeeze().tolist())

  # Optionally, drop the original columns if only tokenized data is needed
  df.drop(columns=[col for col in df.columns if not col.endswith('_input_ids') and not col.endswith('_attention_mask')], inplace=True)

  # Define the columns to keep in the split datasets
  tokenized_columns = [col for col in df.columns if col.endswith('_input_ids') or col.endswith('_attention_mask')]

  # Split the dataset
  train_df, eval_df = train_test_split(df[tokenized_columns], test_size=0.2, random_state=42)

  # Optionally reset the index
  train_df.reset_index(drop=True, inplace=True)
  eval_df.reset_index(drop=True, inplace=True)

  # Save the training and evaluation DataFrames to CSV files
  train_df.to_csv(tokenized_dataset_path + 'train_' + file.split('/')[-1], index=False)
  eval_df.to_csv(tokenized_dataset_path + 'eval_'+ file.split('/')[-1], index=False)

# Tokenizing dataset

In [ ]:
# Load all the dataset
for file_path in all_processed_files_path:
    df = pd.read_csv(file_path)
    if len(df) > 1:
        tokenize_dataset(file_path)

# Load base LLM

In [3]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=True,
)
model = AutoModelForCausalLM.from_pretrained(
          model_name, quantization_config=bnb_config, device_map={"": 0}
)
#Configure the pad token in the model
model.config.pad_token_id = tokenizer.pad_token_id

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

# Initialize streaming class

In [4]:
class StreamCallback:
    def __init__(self, tokenizer, line_length=80):
        self.tokenizer = tokenizer
        self.line_length = line_length
        self.current_line = ""
        self.output = ""

    def process_token(self, token_id):
        token = self.tokenizer.decode(token_id, skip_special_tokens=True)

        # Append token to current line
        if len(self.current_line + token) > self.line_length:
            # If adding this token exceeds the line length, print the current line and start a new one
            self.output += self.current_line.strip() + "\n"
            self.current_line = token
        else:
            self.current_line += token

        # Ensure a space after each token
        if not token.strip():  # Check if the token is a space or empty (e.g., '\n')
            self.current_line = self.current_line.rstrip() + token
        else:
            self.current_line = self.current_line + " "

    def finalize(self):
        if self.current_line:
            self.output += self.current_line.strip() + "\n"
        print(self.output, end='', flush=True)

# Define prompt for loaded LLM

In [5]:
from transformers import GenerationConfig

def generate_with_streaming(instruction, model, tokenizer):
    prompt = "[INST] " + instruction + " [/INST]\n"
    inputs = tokenizer(prompt, return_tensors="pt").to('cuda')
    callback = StreamCallback(tokenizer)

    # Define generation config
    generation_config = GenerationConfig(
        pad_token_id=tokenizer.pad_token_id,
        temperature=1.0,
        top_p=1.0,
        top_k=50,
        num_beams=1,
        max_new_tokens=256
    )

    # Generate tokens one by one
    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]
    generated_ids = input_ids

    model.eval()
    with torch.no_grad():
        for _ in range(generation_config.max_new_tokens):
            outputs = model(
                input_ids=generated_ids,
                attention_mask=attention_mask,
                return_dict=True
            )
            next_token_logits = outputs.logits[:, -1, :]
            next_token_id = torch.argmax(next_token_logits, dim=-1)
            next_token_id = next_token_id.unsqueeze(-1)

            # Process the token
            callback.process_token(next_token_id.item())

            # Append the token to the generated_ids
            generated_ids = torch.cat((generated_ids, next_token_id), dim=1)
            attention_mask = torch.cat((attention_mask, torch.ones_like(next_token_id)), dim=1)

            # Break if end-of-sequence token is generated
            if next_token_id.item() == tokenizer.eos_token_id:
                break

    # Finalize the callback to print any remaining word
    callback.finalize()

# Prompt Demo

In [7]:
# Example usage
instruction = "What do you know about healthcare domain?"
generate_with_streaming(instruction, model, tokenizer)

The healthcare domain refers to the field of medicine and healthcare , which
involves the prevention , diagnosis , and treatment of illness es and diseases .
It en compass es a wide range of discipl ines , including medical science ,
nursing , pharm ac ology , and public health .
In the healthcare domain ,
healthcare providers such as doctors , nurses , and pharm ac ists work together
to provide high - quality care to patients . They use their knowledge and
expertise to diagn ose and treat illness es , as well as to prevent future
health problems .
Health care providers also work to improve the overall health
of the population by promoting healthy behaviors and providing education and
resources to help people manage their health . This can include things like vacc
inations , screen ings , and healthy lifestyle changes .
Over all , the
healthcare domain is an important field that plays a critical role in improving
the health and well - being of individuals and communities .


# Initialization PEFT

In [11]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

# Configuration PEFT

In [12]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32,
    lora_alpha=64,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
        "lm_head",
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 85041152 || all params: 3837112320 || trainable%: 2.2162799758751914


# Train model

In [ ]:
import transformers
from datetime import datetime

project = "A_E_Chatbot"
base_model_name = "mistral"
run_name = base_model_name + "-" + project
output_dir = "./" + run_name

trainer = transformers.Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=transformers.TrainingArguments(
        output_dir=output_dir,
        warmup_steps=1,
        per_device_train_batch_size=4,
        gradient_accumulation_steps=1,
        gradient_checkpointing=True,
        max_steps=500,
        learning_rate=2.5e-4, # Want a small lr for finetuning
        #bf16=True,
        optim="paged_adamw_8bit",
        logging_steps=25,              # When to start reporting loss
        logging_dir="./logs",        # Directory for storing logs
        save_strategy="steps",       # Save the model checkpoint every logging step
        save_steps=25,                # Save checkpoints every 50 steps
        evaluation_strategy="steps", # Evaluate the model every logging step
        eval_steps=25,               # Evaluate and save checkpoints every 50 steps
        do_eval=True,                # Perform evaluation at the end of training
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train(resume_from_checkpoint=True)